#  Análise de Dados: Hábitos e Desempenho Estudantil

##  Estrutura da Análise
1. **Configuração e Carregamento** - Setup inicial e importação dos dados
2. **Exploração Inicial** - Primeira visão dos dados e estrutura
3. **Análise de Qualidade** - Verificação de valores ausentes e inconsistências
4. **Tratamento de Dados** - Limpeza e preparação dos dados
5. **Variáveis Derivadas** - Criação de novas métricas
6. **Visualizações** - Gráficos e análises exploratórias
7. **Insights e Recomendações** - Conclusões e sugestões



In [ ]:
# =============================================================================
# 1. CONFIGURAÇÃO E CARREGAMENTO DE DADOS
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurações para visualizações de alta qualidade
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Carregar os dados
data_path = Path("../../data/habitos_e_desempenho_estudantil.csv")
df = pd.read_csv(data_path)

print(f"Dados carregados: {data_path}")
print(f"Dimensões do dataset: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
print("=" * 60)


In [ ]:
# =============================================================================
# 2. EXPLORAÇÃO INICIAL
# =============================================================================

print("INFORMAÇÕES BÁSICAS DO DATASET")
print("=" * 50)
print(f"• Total de registros: {df.shape[0]:,}")
print(f"• Total de variáveis: {df.shape[1]}")
print(f"• Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Informações sobre tipos de dados
print("\n TIPOS DE DADOS")
print("=" * 30)
print(df.dtypes.value_counts())

# Primeiras linhas
print("\n PRIMEIRAS 5 LINHAS")
print("=" * 30)
display(df.head())

# Informações gerais
print("\n INFORMAÇÕES GERAIS")
print("=" * 30)
df.info()


## 🔍 Exploração Inicial dos Dados

Esta seção fornece uma visão geral dos dados, incluindo:
- **Dimensões do dataset** - Número de registros e variáveis
- **Tipos de dados** - Identificação de variáveis numéricas e categóricas
- **Primeiras observações** - Amostra dos dados para entender a estrutura
- **Informações gerais** - Resumo estatístico básico


In [ ]:
# =============================================================================
# 3. ANÁLISE DE QUALIDADE
#
# 3.1 VALORES AUSENTES
# =============================================================================
print("📊 ANÁLISE DE VALORES AUSENTES")
print("=" * 40)
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Valores_Ausentes': missing_data,
    'Percentual': missing_percent
}).sort_values('Valores_Ausentes', ascending=False)

if missing_df['Valores_Ausentes'].sum() > 0:
    display(missing_df[missing_df['Valores_Ausentes'] > 0])
else:
    print("✅ Nenhum valor ausente encontrado!")

# =============================================================================
# 3.2 DETECÇÃO DE VALORES NEGATIVOS E INCONSISTÊNCIAS
# =============================================================================
print("=" * 60)

# Verificar valores negativos em variáveis que não deveriam ter
numeric_cols = df.select_dtypes(include=[np.number]).columns
negative_values = {}

for col in numeric_cols:
    if col not in ['age']:  # age pode ser negativo em alguns contextos
        neg_count = (df[col] < 0).sum()
        if neg_count > 0:
            negative_values[col] = neg_count
            print(f"⚠️  {col}: {neg_count} valores negativos encontrados")

if not negative_values:
    print("✅ Nenhum valor negativo encontrado!")

# =============================================================================
# 3.3 DETECÇÃO DE OUTLIERS EXTREMOS
# =============================================================================
print("\n DETECÇÃO DE OUTLIERS EXTREMOS")
print("=" * 40)

outliers_summary = {}
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR  # Usando 3*IQR para detectar outliers extremos
    upper_bound = Q3 + 3 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_count = len(outliers)
    
    if outlier_count > 0:
        outliers_summary[col] = {
            'count': outlier_count,
            'percentage': (outlier_count / len(df)) * 100,
            'min_outlier': outliers[col].min(),
            'max_outlier': outliers[col].max()
        }
        print(f"🔍 {col}: {outlier_count} outliers extremos ({outlier_count/len(df)*100:.1f}%)")
        print(f"   Range normal: [{lower_bound:.2f}, {upper_bound:.2f}]")
        print(f"   Outliers: [{outliers[col].min():.2f}, {outliers[col].max():.2f}]")

if not outliers_summary:
    print("✅ Nenhum outlier extremo detectado!")

# =============================================================================
# 3.4 ANÁLISE DE INCONSISTÊNCIAS LÓGICAS
# =============================================================================
print("\n🔍 ANÁLISE DE INCONSISTÊNCIAS LÓGICAS")
print("=" * 50)

# Verificar inconsistências lógicas
inconsistencies = []

# 1. Horas de estudo + sono + lazer > 24h
df['total_daily_hours'] = df['study_hours_per_day'] + df['sleep_hours'] + df['social_media_hours'] + df['netflix_hours']
excessive_hours = df[df['total_daily_hours'] > 24]
if len(excessive_hours) > 0:
    inconsistencies.append(f"⏰ {len(excessive_hours)} registros com mais de 24h de atividades diárias")

# 2. Notas impossíveis (fora do range 0-100)
invalid_scores = df[(df['exam_score'] < 0) | (df['exam_score'] > 100)]
if len(invalid_scores) > 0:
    inconsistencies.append(f"📊 {len(invalid_scores)} registros com notas inválidas (fora de 0-100)")

# 3. Percentual de presença > 100%
invalid_attendance = df[df['attendance_percentage'] > 100]
if len(invalid_attendance) > 0:
    inconsistencies.append(f"📅 {len(invalid_attendance)} registros com presença > 100%")

# 4. Idades inconsistentes
invalid_ages = df[(df['age'] < 16) | (df['age'] > 30)]
if len(invalid_ages) > 0:
    inconsistencies.append(f"👤 {len(invalid_ages)} registros com idades inconsistentes (16-30 anos)")

# 5. Saúde mental fora do range 1-10
invalid_mental_health = df[(df['mental_health_rating'] < 1) | (df['mental_health_rating'] > 10)]
if len(invalid_mental_health) > 0:
    inconsistencies.append(f"🧠 {len(invalid_mental_health)} registros com saúde mental fora do range 1-10")

if inconsistencies:
    print("🚨 INCONSISTÊNCIAS DETECTADAS:")
    for inconsistency in inconsistencies:
        print(f"  {inconsistency}")
else:
    print("✅ Nenhuma inconsistência lógica detectada!")

# =============================================================================
# 3.5 DUPLICATAS E DADOS ÚNICOS
# =============================================================================
print("\n🔄 ANÁLISE DE DUPLICATAS")
print("=" * 30)
duplicates = df.duplicated().sum()
print(f"Duplicatas completas: {duplicates}")

# Verificar duplicatas por ID
duplicate_ids = df['student_id'].duplicated().sum()
print(f"IDs duplicados: {duplicate_ids}")

print("\n🎯 VALORES ÚNICOS POR COLUNA")
print("=" * 40)
unique_counts = df.nunique()
for col in df.columns:
    print(f"{col}: {unique_counts[col]} valores únicos")

# =============================================================================
# 3.6 RESUMO DE QUALIDADE DOS DADOS
# =============================================================================
print("\n📋 RESUMO DE QUALIDADE DOS DADOS")
print("=" * 50)
print(f"• Total de registros: {len(df):,}")
print(f"• Total de variáveis: {len(df.columns)}")
print(f"• Valores ausentes: {missing_df['Valores_Ausentes'].sum()}")
print(f"• Valores negativos inesperados: {sum(negative_values.values())}")
print(f"• Outliers extremos: {sum([info['count'] for info in outliers_summary.values()])}")
print(f"• Inconsistências lógicas: {len(inconsistencies)}")
print(f"• Duplicatas: {duplicates}")

# Calcular score de qualidade
quality_score = 100
if missing_df['Valores_Ausentes'].sum() > 0:
    quality_score -= 10
if negative_values:
    quality_score -= 15
if outliers_summary:
    quality_score -= 10
if inconsistencies:
    quality_score -= 20
if duplicates > 0:
    quality_score -= 5

print(f"\n SCORE DE QUALIDADE DOS DADOS: {quality_score}/100")

print("\n📈 ESTATÍSTICAS DESCRITIVAS")
print("=" * 40)
display(df.describe())


## 🔧 Análise de Qualidade dos Dados

Esta seção identifica e trata problemas de qualidade nos dados:

### 📊 Verificações Realizadas:
- **Valores Ausentes** - Identificação de campos vazios
- **Valores Negativos** - Detecção de inconsistências lógicas
- **Outliers Extremos** - Identificação de valores atípicos
- **Duplicatas** - Verificação de registros duplicados
- **Inconsistências Lógicas** - Validação de regras de negócio


In [ ]:
# =============================================================================
# 4. TRATAMENTO DE DADOS INCONSISTENTES
# =============================================================================

print(" TRATAMENTO DE DADOS INCONSISTENTES")
print("=" * 50)

# Criar cópia para tratamento
df_clean = df.copy()

# =============================================================================
# 4.1 TRATAMENTO DE OUTLIERS EXTREMOS
# =============================================================================
print(" TRATAMENTO DE OUTLIERS EXTREMOS")
print("=" * 40)

# Função para tratar outliers usando winsorization
def winsorize_outliers(series, lower_percentile=1, upper_percentile=99):
    """Aplica winsorization para tratar outliers extremos"""
    lower_bound = series.quantile(lower_percentile / 100)
    upper_bound = series.quantile(upper_percentile / 100)
    
    # Contar outliers antes do tratamento
    outliers_before = ((series < lower_bound) | (series > upper_bound)).sum()
    
    # Aplicar winsorization
    series_clean = series.clip(lower=lower_bound, upper=upper_bound)
    
    return series_clean, outliers_before

# Tratar outliers em variáveis numéricas importantes
numeric_vars_to_clean = ['study_hours_per_day', 'social_media_hours', 'netflix_hours', 
                        'attendance_percentage', 'sleep_hours', 'exam_score']

outliers_treated = {}
for var in numeric_vars_to_clean:
    if var in df_clean.columns:
        df_clean[var], outliers_count = winsorize_outliers(df_clean[var])
        if outliers_count > 0:
            outliers_treated[var] = outliers_count
            print(f"🔧 {var}: {outliers_count} outliers tratados")

if outliers_treated:
    print(f"\n✅ Total de outliers tratados: {sum(outliers_treated.values())}")
else:
    print("✅ Nenhum outlier extremo encontrado para tratamento!")

# =============================================================================
# 4.2 TRATAMENTO DE INCONSISTÊNCIAS LÓGICAS
# =============================================================================
print("\n🔍 TRATAMENTO DE INCONSISTÊNCIAS LÓGICAS")
print("=" * 50)

# 1. Corrigir horas totais excessivas (cap em 24h)
df_clean['total_daily_hours'] = (df_clean['study_hours_per_day'] + 
                                df_clean['sleep_hours'] + 
                                df_clean['social_media_hours'] + 
                                df_clean['netflix_hours'])

excessive_hours = df_clean[df_clean['total_daily_hours'] > 24]
if len(excessive_hours) > 0:
    print(f"⏰ Ajustando {len(excessive_hours)} registros com horas excessivas")
    # Reduzir proporcionalmente as atividades
    for idx in excessive_hours.index:
        total = df_clean.loc[idx, 'total_daily_hours']
        scale_factor = 24 / total
        df_clean.loc[idx, 'study_hours_per_day'] *= scale_factor
        df_clean.loc[idx, 'social_media_hours'] *= scale_factor
        df_clean.loc[idx, 'netflix_hours'] *= scale_factor

# 2. Corrigir notas fora do range 0-100
invalid_scores = df_clean[(df_clean['exam_score'] < 0) | (df_clean['exam_score'] > 100)]
if len(invalid_scores) > 0:
    print(f"📊 Corrigindo {len(invalid_scores)} notas inválidas")
    df_clean['exam_score'] = df_clean['exam_score'].clip(0, 100)

# 3. Corrigir presença > 100%
invalid_attendance = df_clean[df_clean['attendance_percentage'] > 100]
if len(invalid_attendance) > 0:
    print(f"📅 Corrigindo {len(invalid_attendance)} presenças inválidas")
    df_clean['attendance_percentage'] = df_clean['attendance_percentage'].clip(0, 100)

# 4. Corrigir idades inconsistentes
invalid_ages = df_clean[(df_clean['age'] < 16) | (df_clean['age'] > 30)]
if len(invalid_ages) > 0:
    print(f"👤 Corrigindo {len(invalid_ages)} idades inconsistentes")
    df_clean['age'] = df_clean['age'].clip(16, 30)

# 5. Corrigir saúde mental fora do range 1-10
invalid_mental_health = df_clean[(df_clean['mental_health_rating'] < 1) | (df_clean['mental_health_rating'] > 10)]
if len(invalid_mental_health) > 0:
    print(f"🧠 Corrigindo {len(invalid_mental_health)} avaliações de saúde mental inválidas")
    df_clean['mental_health_rating'] = df_clean['mental_health_rating'].clip(1, 10)

# =============================================================================
# 4.3 TRATAMENTO DE VALORES AUSENTES
# =============================================================================
print("\n📊 TRATAMENTO DE VALORES AUSENTES")
print("=" * 40)

# Verificar valores ausentes após limpeza
missing_after_clean = df_clean.isnull().sum()
if missing_after_clean.sum() > 0:
    print("Valores ausentes encontrados:")
    for col, count in missing_after_clean[missing_after_clean > 0].items():
        print(f"  {col}: {count} valores ausentes")
        
        # Estratégias de imputação baseadas no tipo de variável
        if col in ['parental_education_level']:
            # Para variáveis categóricas, usar moda
            mode_value = df_clean[col].mode()[0]
            df_clean[col].fillna(mode_value, inplace=True)
            print(f"    → Preenchido com moda: {mode_value}")
        elif col in ['age', 'exercise_frequency', 'mental_health_rating']:
            # Para variáveis numéricas discretas, usar mediana
            median_value = df_clean[col].median()
            df_clean[col].fillna(median_value, inplace=True)
            print(f"    → Preenchido com mediana: {median_value}")
        else:
            # Para outras variáveis numéricas, usar média
            mean_value = df_clean[col].mean()
            df_clean[col].fillna(mean_value, inplace=True)
            print(f"    → Preenchido com média: {mean_value:.2f}")
else:
    print("✅ Nenhum valor ausente encontrado!")

# =============================================================================
# 4.4 VERIFICAÇÃO FINAL DE QUALIDADE
# =============================================================================
print("\n✅ VERIFICAÇÃO FINAL DE QUALIDADE")
print("=" * 40)

# Recalcular métricas de qualidade
final_missing = df_clean.isnull().sum().sum()
final_duplicates = df_clean.duplicated().sum()
final_negative = (df_clean.select_dtypes(include=[np.number]) < 0).sum().sum()

print(f"• Valores ausentes: {final_missing}")
print(f"• Duplicatas: {final_duplicates}")
print(f"• Valores negativos: {final_negative}")

# Verificar se ainda há inconsistências
final_inconsistencies = []
final_inconsistencies.extend(df_clean[df_clean['exam_score'] < 0].index.tolist())
final_inconsistencies.extend(df_clean[df_clean['exam_score'] > 100].index.tolist())
final_inconsistencies.extend(df_clean[df_clean['attendance_percentage'] > 100].index.tolist())

print(f"• Inconsistências restantes: {len(set(final_inconsistencies))}")


## 🛠️ Tratamento de Dados Inconsistentes

Esta seção aplica correções e limpezas nos dados identificados:

### 🔄 Processos de Limpeza:
- **Tratamento de Valores Ausentes** - Preenchimento ou remoção de dados faltantes
- **Correção de Inconsistências** - Ajuste de valores que não fazem sentido
- **Padronização** - Uniformização de formatos e categorias
- **Validação Final** - Verificação da qualidade após tratamento


In [ ]:
# =============================================================================
# 5. COMPARATIVO VISUAL: ANTES vs DEPOIS DO TRATAMENTO
# =============================================================================

print("📊 COMPARATIVO VISUAL: ANTES vs DEPOIS DO TRATAMENTO")
print("=" * 60)

# Configurar subplots para comparação
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Comparativo: Dados Originais vs Dados Tratados', fontsize=16, fontweight='bold')

# Variáveis principais para comparação
main_vars = ['study_hours_per_day', 'social_media_hours', 'netflix_hours', 
            'attendance_percentage', 'sleep_hours', 'exam_score']

# Cores para distinguir antes/depois
colors_before = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4', '#feca57', '#ff9ff3']
colors_after = ['#ff8e8e', '#6ed5c7', '#6bc5d8', '#a8d5c4', '#fed766', '#ffb3f3']

for i, var in enumerate(main_vars):
    row = i // 3
    col = i % 3
    ax = axes[row, col]
    
    # Dados originais
    original_data = df[var].dropna()
    treated_data = df_clean[var].dropna()
    
    # Criar histogramas sobrepostos
    ax.hist(original_data, bins=30, alpha=0.7, label='Antes', 
            color=colors_before[i], density=True)
    ax.hist(treated_data, bins=30, alpha=0.7, label='Depois', 
            color=colors_after[i], density=True)
    
    # Configurações do gráfico
    ax.set_title(f'{var.replace("_", " ").title()}', fontweight='bold')
    ax.set_xlabel('Valor')
    ax.set_ylabel('Densidade')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Adicionar estatísticas
    orig_mean = original_data.mean()
    treat_mean = treated_data.mean()
    orig_std = original_data.std()
    treat_std = treated_data.std()
    
    ax.text(0.02, 0.98, f'Antes: μ={orig_mean:.2f}, σ={orig_std:.2f}', 
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.text(0.02, 0.88, f'Depois: μ={treat_mean:.2f}, σ={treat_std:.2f}', 
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# =============================================================================
# 5.1 ANÁLISE ESTATÍSTICA DO TRATAMENTO
# =============================================================================

print("\n📈 ANÁLISE ESTATÍSTICA DO TRATAMENTO")
print("=" * 50)

# Criar DataFrame comparativo
comparison_stats = []

for var in main_vars:
    if var in df.columns and var in df_clean.columns:
        orig_data = df[var].dropna()
        treat_data = df_clean[var].dropna()
        
        stats = {
            'Variável': var.replace('_', ' ').title(),
            'Antes_Media': orig_data.mean(),
            'Depois_Media': treat_data.mean(),
            'Antes_Desvio': orig_data.std(),
            'Depois_Desvio': treat_data.std(),
            'Antes_Min': orig_data.min(),
            'Depois_Min': treat_data.min(),
            'Antes_Max': orig_data.max(),
            'Depois_Max': treat_data.max(),
            'Diferenca_Media': treat_data.mean() - orig_data.mean(),
            'Diferenca_Desvio': treat_data.std() - orig_data.std()
        }
        comparison_stats.append(stats)

comparison_df = pd.DataFrame(comparison_stats)

# Exibir tabela comparativa
print("📊 ESTATÍSTICAS COMPARATIVAS")
print("=" * 40)
display(comparison_df.round(3))

# =============================================================================
# 5.2 ANÁLISE DE IMPACTO DO TRATAMENTO
# =============================================================================

print("\n🎯 ANÁLISE DE IMPACTO DO TRATAMENTO")
print("=" * 45)

# Calcular métricas de impacto
impact_analysis = []

for var in main_vars:
    if var in df.columns and var in df_clean.columns:
        orig_data = df[var].dropna()
        treat_data = df_clean[var].dropna()
        
        # Métricas de impacto
        mean_change = ((treat_data.mean() - orig_data.mean()) / orig_data.mean()) * 100
        std_change = ((treat_data.std() - orig_data.std()) / orig_data.std()) * 100
        range_change = ((treat_data.max() - treat_data.min()) - (orig_data.max() - orig_data.min())) / (orig_data.max() - orig_data.min()) * 100
        
        impact_analysis.append({
            'Variável': var.replace('_', ' ').title(),
            'Mudança_Média_%': mean_change,
            'Mudança_Desvio_%': std_change,
            'Mudança_Range_%': range_change,
            'Impacto': 'Alto' if abs(mean_change) > 5 else 'Baixo' if abs(mean_change) < 1 else 'Moderado'
        })

impact_df = pd.DataFrame(impact_analysis)
display(impact_df.round(2))

# =============================================================================
# 5.3 VISUALIZAÇÃO DE BOXPLOTS COMPARATIVOS
# =============================================================================

print("\n📦 BOXPLOTS COMPARATIVOS")
print("=" * 30)

# Criar boxplots comparativos
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Boxplots Comparativos: Antes vs Depois do Tratamento', fontsize=16, fontweight='bold')

for i, var in enumerate(main_vars):
    row = i // 3
    col = i % 3
    ax = axes[row, col]
    
    # Preparar dados para boxplot
    data_to_plot = [df[var].dropna(), df_clean[var].dropna()]
    labels = ['Antes', 'Depois']
    
    # Criar boxplot
    bp = ax.boxplot(data_to_plot, labels=labels, patch_artist=True)
    
    # Colorir as caixas
    bp['boxes'][0].set_facecolor('#ff6b6b')
    bp['boxes'][1].set_facecolor('#4ecdc4')
    
    # Configurações
    ax.set_title(f'{var.replace("_", " ").title()}', fontweight='bold')
    ax.set_ylabel('Valor')
    ax.grid(True, alpha=0.3)
    
    # Adicionar estatísticas no gráfico
    orig_q1, orig_q3 = df[var].quantile([0.25, 0.75])
    treat_q1, treat_q3 = df_clean[var].quantile([0.25, 0.75])
    
    ax.text(0.02, 0.98, f'Q1-Q3 Antes: {orig_q1:.1f}-{orig_q3:.1f}', 
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.text(0.02, 0.88, f'Q1-Q3 Depois: {treat_q1:.1f}-{treat_q3:.1f}', 
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# =============================================================================
# 5.4 RESUMO DO TRATAMENTO
# =============================================================================

print("\n📋 RESUMO DO TRATAMENTO REALIZADO")
print("=" * 45)

# Contar mudanças significativas
significant_changes = impact_df[impact_df['Mudança_Média_%'].abs() > 1]
moderate_changes = impact_df[(impact_df['Mudança_Média_%'].abs() > 0.5) & (impact_df['Mudança_Média_%'].abs() <= 1)]
minimal_changes = impact_df[impact_df['Mudança_Média_%'].abs() <= 0.5]

print(f"🔍 Mudanças Significativas (>1%): {len(significant_changes)} variáveis")
print(f"🔍 Mudanças Moderadas (0.5-1%): {len(moderate_changes)} variáveis")
print(f"🔍 Mudanças Mínimas (<0.5%): {len(minimal_changes)} variáveis")

print(f"\n✅ Tratamento concluído com sucesso!")
print(f"📊 Total de variáveis analisadas: {len(main_vars)}")
print(f"📈 Dados originais preservados: {len(df):,} registros")
print(f"📈 Dados tratados: {len(df_clean):,} registros")



## 📈 Criação de Variáveis Derivadas

Esta seção cria novas métricas e indicadores para enriquecer a análise:

### 🆕 Novas Variáveis Criadas:
- **Índices Compostos** - Métricas que combinam múltiplas variáveis
- **Categorizações** - Segmentação de variáveis contínuas
- **Ratios e Proporções** - Relações entre diferentes métricas
- **Indicadores de Risco** - Flags para identificar estudantes em risco


In [ ]:
# =============================================================================
# 5. VARIÁVEIS DERIVADAS
# =============================================================================

print("VARIÁVEIS DERIVADAS")
print("=" * 50)

# Usar o dataset limpo como base
df_processed = df_clean.copy()

# =============================================================================
# 5.1 VARIÁVEIS CATEGÓRICAS
# =============================================================================

# Funções de categorização
def categorize_social_media_usage(hours):
    if hours <= 1: return 'Baixo'
    elif hours <= 3: return 'Moderado'
    elif hours <= 5: return 'Alto'
    else: return 'Muito Alto'

def categorize_study_hours(hours):
    if hours <= 2: return 'Baixo'
    elif hours <= 4: return 'Moderado'
    elif hours <= 6: return 'Alto'
    else: return 'Muito Alto'

def categorize_sleep_hours(hours):
    if hours < 6: return 'Insuficiente'
    elif hours <= 7: return 'Adequado'
    elif hours <= 9: return 'Ideal'
    else: return 'Excessivo'

def categorize_academic_performance(score):
    if score < 50: return 'Baixo'
    elif score < 70: return 'Regular'
    elif score < 85: return 'Bom'
    else: return 'Excelente'

# Aplicar categorizações
df_processed['social_media_category'] = df_processed['social_media_hours'].apply(categorize_social_media_usage)
df_processed['study_hours_category'] = df_processed['study_hours_per_day'].apply(categorize_study_hours)
df_processed['sleep_category'] = df_processed['sleep_hours'].apply(categorize_sleep_hours)
df_processed['performance_category'] = df_processed['exam_score'].apply(categorize_academic_performance)

# =============================================================================
# 5.2 VARIÁVEIS NUMÉRICAS DERIVADAS
# =============================================================================

# Índice de bem-estar
df_processed['wellness_index'] = (
    df_processed['sleep_hours'] * 0.4 + 
    df_processed['exercise_frequency'] * 0.3 + 
    df_processed['mental_health_rating'] * 0.3
)

# Horas totais de lazer
df_processed['total_leisure_hours'] = (
    df_processed['social_media_hours'] + 
    df_processed['netflix_hours']
)

# Tempo disponível para estudo
df_processed['available_study_time'] = (
    24 - 
    df_processed['sleep_hours'] - 
    df_processed['total_leisure_hours'] - 
    df_processed['part_time_job'].map({'Yes': 8, 'No': 0})
)

# Razão estudo/entretenimento
df_processed['study_screen_ratio'] = (
    df_processed['study_hours_per_day'] / 
    (df_processed['total_leisure_hours'] + 0.1)  # +0.1 para evitar divisão por zero
)

# Flag de underperformers
df_processed['underperformer_flag'] = (
    ((df_processed['attendance_percentage'] > 80) & (df_processed['exam_score'] < 60)) |
    ((df_processed['study_hours_per_day'] > 4) & (df_processed['exam_score'] < 50))
)

print("✅ Variáveis derivadas criadas com sucesso!")
print("📊 Total de variáveis no dataset: {df_processed.shape[1]}")
print("📈 Novas variáveis adicionadas: 9")

# =============================================================================
# 5.3 ANÁLISE DAS NOVAS VARIÁVEIS
# =============================================================================

# Estatísticas das novas variáveis numéricas
print("\n📈 ESTATÍSTICAS DAS NOVAS VARIÁVEIS NUMÉRICAS")
print("=" * 60)

new_numeric_vars = ['total_leisure_hours', 'available_study_time', 'study_screen_ratio', 'wellness_index']
for var in new_numeric_vars:
    print(f"\n{var}:")
    print(f"  • Média: {df_processed[var].mean():.2f}")
    print(f"  • Mediana: {df_processed[var].median():.2f}")
    print(f"  • Desvio Padrão: {df_processed[var].std():.2f}")

# Análise dos underperformers
underperformers_count = df_processed['underperformer_flag'].sum()
print(f"\n🎯 UNDERPERFORMERS IDENTIFICADOS: {underperformers_count} ({underperformers_count/len(df_processed)*100:.1f}%)")

# Correlações das novas variáveis
print("\n🔍 CORRELAÇÕES COM DESEMPENHO ACADÊMICO")
print("=" * 60)
for var in new_numeric_vars:
    corr = df_processed[var].corr(df_processed['exam_score'])
    direction = "📈" if corr > 0 else "📉"
    strength = "Forte" if abs(corr) > 0.5 else "Moderada" if abs(corr) > 0.3 else "Fraca"
    print(f"{direction} {var}: {corr:.3f} ({strength})")


## 📊 Visualizações e Análises Exploratórias

Esta seção apresenta gráficos e análises visuais dos dados:

### 🎨 Tipos de Visualizações:
- **Distribuições** - Histogramas e gráficos de densidade
- **Correlações** - Mapas de calor e matrizes de correlação
- **Comparações** - Gráficos de barras e boxplots
- **Tendências** - Análises temporais e padrões comportamentais


In [ ]:
# =============================================================================
# 6. VISUALIZAÇÕES
# =============================================================================

print("📊 VISUALIZAÇÕES")
print("=" * 60)

# Configurações avançadas para visualizações
plt.style.use('default')
sns.set_style("whitegrid")
plt.rcParams.update({
    'figure.figsize': (16, 10),
    'font.size': 12,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.3
})

# Paleta de cores profissional
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#7209B7', '#F77F00', '#06FFA5']
sns.set_palette(colors)

# =============================================================================
# 6.1 VISUALIZAÇÃO PRINCIPAL
# =============================================================================

fig = plt.figure(figsize=(20, 16))
gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)

# 1. Distribuição de Desempenho (Gráfico Principal)
ax1 = fig.add_subplot(gs[0, :2])
performance_counts = df_processed['performance_category'].value_counts()
wedges, texts, autotexts = ax1.pie(performance_counts.values, 
                                  labels=performance_counts.index,
                                  autopct='%1.1f%%',
                                  startangle=90,
                                  colors=colors[:len(performance_counts)],
                                  explode=[0.05 if x == performance_counts.max() else 0 for x in performance_counts.values])
ax1.set_title('Distribuição de Desempenho Acadêmico', fontsize=18, fontweight='bold', pad=20)
ax1.axis('equal')

# 2. Correlação com Desempenho
ax2 = fig.add_subplot(gs[0, 2:])
numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
corr_matrix = df_processed[numeric_cols].corr()
exam_corr = corr_matrix['exam_score'].drop('exam_score').sort_values(ascending=True)

bars = ax2.barh(range(len(exam_corr)), exam_corr.values, 
                color=[colors[0] if x > 0 else colors[3] for x in exam_corr.values])
ax2.set_yticks(range(len(exam_corr)))
ax2.set_yticklabels([col.replace('_', ' ').title() for col in exam_corr.index])
ax2.set_xlabel('Correlação com Desempenho', fontsize=12)
ax2.set_title('Fatores que Influenciam o Desempenho', fontsize=16, fontweight='bold')
ax2.axvline(x=0, color='black', linestyle='-', alpha=0.3)
ax2.grid(True, alpha=0.3)

# Adicionar valores nas barras
for i, (bar, value) in enumerate(zip(bars, exam_corr.values)):
    ax2.text(value + (0.01 if value > 0 else -0.01), bar.get_y() + bar.get_height()/2, 
             f'{value:.3f}', ha='left' if value > 0 else 'right', va='center', fontweight='bold')

# 3. Análise de Horas de Estudo vs Desempenho (Scatter com Regressão)
ax3 = fig.add_subplot(gs[1, :2])
scatter = ax3.scatter(df_processed['study_hours_per_day'], df_processed['exam_score'], 
                     c=df_processed['mental_health_rating'], 
                     cmap='viridis', alpha=0.7, s=60, edgecolors='white', linewidth=0.5)

# Linha de regressão
z = np.polyfit(df_processed['study_hours_per_day'], df_processed['exam_score'], 1)
p = np.poly1d(z)
ax3.plot(df_processed['study_hours_per_day'], p(df_processed['study_hours_per_day']), 
         "r--", alpha=0.8, linewidth=3, label=f'Tendência (R²={np.corrcoef(df_processed["study_hours_per_day"], df_processed["exam_score"])[0,1]**2:.3f})')

ax3.set_xlabel('Horas de Estudo por Dia', fontsize=12)
ax3.set_ylabel('Nota do Exame', fontsize=12)
ax3.set_title('Relação entre Horas de Estudo e Desempenho\n(Cor: Saúde Mental)', fontsize=14, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Colorbar para saúde mental
cbar = plt.colorbar(scatter, ax=ax3, shrink=0.8)
cbar.set_label('Saúde Mental (1-10)', fontsize=10)

# 4. Boxplot Comparativo por Categorias
ax4 = fig.add_subplot(gs[1, 2:])
categories = ['social_media_category', 'sleep_category', 'diet_quality']
category_data = []
category_labels = []

for cat in categories:
    if cat in df_processed.columns:
        for value in df_processed[cat].unique():
            subset = df_processed[df_processed[cat] == value]
            category_data.append(subset['exam_score'].values)
            category_labels.append(f"{cat.replace('_', ' ').title()}\n{value}")

box_plot = ax4.boxplot(category_data, labels=category_labels, patch_artist=True)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax4.set_ylabel('Nota do Exame', fontsize=12)
ax4.set_title('Desempenho por Categorias de Hábitos', fontsize=14, fontweight='bold')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

# 5. Análise Temporal/Sequencial (se houver dados temporais)
ax5 = fig.add_subplot(gs[2, :2])
# Criar um histograma 2D mostrando a distribuição de horas de estudo vs redes sociais
ax5.hist2d(df_processed['study_hours_per_day'], df_processed['social_media_hours'], 
          bins=20, cmap='Blues', alpha=0.8)
ax5.set_xlabel('Horas de Estudo por Dia', fontsize=12)
ax5.set_ylabel('Horas de Redes Sociais', fontsize=12)
ax5.set_title('Distribuição: Estudo vs Redes Sociais', fontsize=14, fontweight='bold')

# Adicionar contornos
from scipy.stats import gaussian_kde
xy = np.vstack([df_processed['study_hours_per_day'], df_processed['social_media_hours']])
density = gaussian_kde(xy)
x_range = np.linspace(df_processed['study_hours_per_day'].min(), df_processed['study_hours_per_day'].max(), 50)
y_range = np.linspace(df_processed['social_media_hours'].min(), df_processed['social_media_hours'].max(), 50)
X, Y = np.meshgrid(x_range, y_range)
positions = np.vstack([X.ravel(), Y.ravel()])
Z = np.reshape(density(positions).T, X.shape)
ax5.contour(X, Y, Z, levels=5, colors='red', alpha=0.6, linewidths=2)

# 6. Métricas de Performance (Gauge-style)
ax6 = fig.add_subplot(gs[2, 2:])
ax6.axis('off')

# Calcular métricas principais
high_performers = len(df_processed[df_processed['performance_category'] == 'Excelente'])
avg_score = df_processed['exam_score'].mean()
study_correlation = df_processed['study_hours_per_day'].corr(df_processed['exam_score'])

# Criar um "dashboard" com métricas
metrics_text = f"""
📊 MÉTRICAS PRINCIPAIS

🎯 Estudantes de Alto Desempenho
{high_performers} ({high_performers/len(df_processed)*100:.1f}%)

📈 Nota Média Geral
{avg_score:.1f}/100

🔗 Correlação Estudo-Desempenho
{study_correlation:.3f}

⚡ Fator Mais Importante
Horas de Estudo
"""

ax6.text(0.1, 0.9, metrics_text, transform=ax6.transAxes, fontsize=14,
         verticalalignment='top', bbox=dict(boxstyle="round,pad=0.5", facecolor='lightblue', alpha=0.8))

plt.suptitle(' ANÁLISE DE HÁBITOS X DESEMPENHO', 
             fontsize=20, fontweight='bold', y=0.98)
plt.show()

# =============================================================================
# 6.2 VISUALIZAÇÃO DE CORRELAÇÕES AVANÇADA
# =============================================================================

print("\n📈 MAPA DE CALOR DE CORRELAÇÕES")
print("=" * 50)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Mapa de calor completo com máscara
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, 
            mask=mask,
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            square=True,
            fmt='.2f',
            cbar_kws={"shrink": .8},
            ax=ax1,
            linewidths=0.5)
ax1.set_title('Matriz de Correlações Completa', fontsize=16, fontweight='bold')

# Mapa de calor focado no desempenho
exam_corr_data = corr_matrix[['exam_score']].sort_values('exam_score', ascending=False)
sns.heatmap(exam_corr_data, 
            annot=True, 
            cmap='RdYlGn', 
            center=0,
            fmt='.3f',
            cbar_kws={"shrink": .8},
            ax=ax2,
            linewidths=0.5)
ax2.set_title('Correlações com Desempenho Acadêmico', fontsize=16, fontweight='bold')
ax2.set_xlabel('')

plt.tight_layout()
plt.show()

# =============================================================================
# 6.3 ANÁLISE DE PERFIS DE ESTUDANTES
# =============================================================================

print("\n👥 ANÁLISE DE PERFIS DE ESTUDANTES")
print("=" * 50)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Distribuição por Gênero
gender_performance = df_processed.groupby(['gender', 'performance_category']).size().unstack(fill_value=0)
gender_performance_pct = gender_performance.div(gender_performance.sum(axis=1), axis=0) * 100

gender_performance_pct.plot(kind='bar', ax=axes[0,0], color=colors[:4])
axes[0,0].set_title('Distribuição de Desempenho por Gênero', fontweight='bold')
axes[0,0].set_xlabel('Gênero')
axes[0,0].set_ylabel('Percentual (%)')
axes[0,0].legend(title='Desempenho', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0,0].tick_params(axis='x', rotation=0)

# 2. Análise de Idade vs Desempenho
age_bins = pd.cut(df_processed['age'], bins=5, labels=['16-18', '19-21', '22-24', '25-27', '28-30'])
df_processed['age_group'] = age_bins
age_performance = df_processed.groupby(['age_group', 'performance_category']).size().unstack(fill_value=0)
age_performance_pct = age_performance.div(age_performance.sum(axis=1), axis=0) * 100

age_performance_pct.plot(kind='bar', ax=axes[0,1], color=colors[:4])
axes[0,1].set_title('Distribuição de Desempenho por Faixa Etária', fontweight='bold')
axes[0,1].set_xlabel('Faixa Etária')
axes[0,1].set_ylabel('Percentual (%)')
axes[0,1].legend(title='Desempenho', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Análise de Educação dos Pais
parent_education_performance = df_processed.groupby(['parental_education_level', 'performance_category']).size().unstack(fill_value=0)
parent_education_performance_pct = parent_education_performance.div(parent_education_performance.sum(axis=1), axis=0) * 100

parent_education_performance_pct.plot(kind='bar', ax=axes[1,0], color=colors[:4])
axes[1,0].set_title('Distribuição de Desempenho por Educação dos Pais', fontweight='bold')
axes[1,0].set_xlabel('Educação dos Pais')
axes[1,0].set_ylabel('Percentual (%)')
axes[1,0].legend(title='Desempenho', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Análise de Participação Extracurricular
extracurricular_performance = df_processed.groupby(['extracurricular_participation', 'performance_category']).size().unstack(fill_value=0)
extracurricular_performance_pct = extracurricular_performance.div(extracurricular_performance.sum(axis=1), axis=0) * 100

extracurricular_performance_pct.plot(kind='bar', ax=axes[1,1], color=colors[:4])
axes[1,1].set_title('Distribuição de Desempenho por Participação Extracurricular', fontweight='bold')
axes[1,1].set_xlabel('Participação Extracurricular')
axes[1,1].set_ylabel('Percentual (%)')
axes[1,1].legend(title='Desempenho', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1,1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()


## 💡 Insights e Recomendações

Esta seção apresenta as principais descobertas e sugestões baseadas na análise:

### 🎯 Principais Descobertas:
- **Fatores de Impacto** - Variáveis que mais influenciam o desempenho
- **Perfis de Estudantes** - Segmentação por comportamentos e resultados
- **Padrões de Risco** - Identificação de estudantes em situação vulnerável
- **Oportunidades de Melhoria** - Áreas com potencial de intervenção


In [ ]:
# =============================================================================
# 8. INSIGHTS E RECOMENDAÇÕES 
# =============================================================================

print("💡 INSIGHTS E RECOMENDAÇÕES AVANÇADAS")
print("=" * 60)

# =============================================================================
# 8.1 ANÁLISE DE SEGMENTAÇÃO DE ESTUDANTES
# =============================================================================
print("🎯 SEGMENTAÇÃO DE ESTUDANTES POR PERFORMANCE")
print("=" * 60)

# Definir segmentos baseados no desempenho
def segment_students(score):
    if score >= 85:
        return "Excelente"
    elif score >= 70:
        return "Bom"
    elif score >= 50:
        return "Regular"
    else:
        return "Baixo"

df_processed['performance_segment'] = df_processed['exam_score'].apply(segment_students)

# Análise por segmento
segment_analysis = df_processed.groupby('performance_segment').agg({
    'study_hours_per_day': 'mean',
    'social_media_hours': 'mean',
    'sleep_hours': 'mean',
    'mental_health_rating': 'mean',
    'exercise_frequency': 'mean',
    'attendance_percentage': 'mean',
    'exam_score': ['count', 'mean', 'std']
}).round(2)

print("📊 CARACTERÍSTICAS POR SEGMENTO DE PERFORMANCE:")
print("-" * 60)
for segment in ['Excelente', 'Bom', 'Regular', 'Baixo']:
    if segment in segment_analysis.index:
        data = segment_analysis.loc[segment]
        count = data[('exam_score', 'count')]
        mean_score = data[('exam_score', 'mean')]
        std_score = data[('exam_score', 'std')]
        
        print(f"\n🎯 SEGMENTO {segment.upper()}:")
        print(f"  • Quantidade: {count} estudantes ({count/len(df_processed)*100:.1f}%)")
        print(f"  • Nota média: {mean_score:.1f} ± {std_score:.1f}")
        print(f"  • Horas de estudo: {data[('study_hours_per_day', 'mean')]:.1f}h/dia")
        print(f"  • Redes sociais: {data[('social_media_hours', 'mean')]:.1f}h/dia")
        print(f"  • Sono: {data[('sleep_hours', 'mean')]:.1f}h")
        print(f"  • Saúde mental: {data[('mental_health_rating', 'mean')]:.1f}/10")
        print(f"  • Exercício: {data[('exercise_frequency', 'mean')]:.1f}")
        print(f"  • Presença: {data[('attendance_percentage', 'mean')]:.1f}%")

# =============================================================================
# 8.2 ANÁLISE DE FATORES DE RISCO
# =============================================================================
print("\n⚠️  ANÁLISE DE FATORES DE RISCO")
print("=" * 50)

# Identificar estudantes em risco
risk_factors = []
risk_factors.extend(df_processed[df_processed['study_hours_per_day'] < 2].index.tolist())
risk_factors.extend(df_processed[df_processed['mental_health_rating'] < 4].index.tolist())
risk_factors.extend(df_processed[df_processed['attendance_percentage'] < 70].index.tolist())
risk_factors.extend(df_processed[df_processed['social_media_hours'] > 6].index.tolist())

# Contar fatores de risco por estudante
risk_count = pd.Series(risk_factors).value_counts()
high_risk_students = risk_count[risk_count >= 2].index

print(f"🚨 Estudantes com múltiplos fatores de risco: {len(high_risk_students)}")
print(f"📊 Distribuição de fatores de risco:")
print(f"  • 0 fatores: {len(df_processed) - len(risk_count)} estudantes")
print(f"  • 1 fator: {len(risk_count[risk_count == 1])} estudantes")
print(f"  • 2+ fatores: {len(high_risk_students)} estudantes")

# Análise dos estudantes de alto risco
if len(high_risk_students) > 0:
    high_risk_data = df_processed.loc[high_risk_students]
    print(f"\n🔍 CARACTERÍSTICAS DOS ESTUDANTES DE ALTO RISCO:")
    print(f"  • Nota média: {high_risk_data['exam_score'].mean():.1f}")
    print(f"  • Horas de estudo: {high_risk_data['study_hours_per_day'].mean():.1f}h/dia")
    print(f"  • Saúde mental: {high_risk_data['mental_health_rating'].mean():.1f}/10")

# =============================================================================
# 8.3 RECOMENDAÇÕES PERSONALIZADAS
# =============================================================================
print("\n💡 RECOMENDAÇÕES PERSONALIZADAS POR SEGMENTO")
print("=" * 60)

recommendations = {
    'Excelente': [
        "🎯 Manter estratégias atuais de estudo",
        "📚 Explorar desafios acadêmicos adicionais",
        "🤝 Mentorear estudantes com dificuldades",
        "🏆 Participar de competições acadêmicas"
    ],
    'Bom': [
        "📈 Aumentar horas de estudo em 1-2h/dia",
        "🧠 Melhorar técnicas de estudo",
        "⏰ Otimizar gestão de tempo",
        "🎯 Definir metas específicas de melhoria"
    ],
    'Regular': [
        "📚 Implementar rotina de estudo estruturada",
        "🧘 Praticar técnicas de relaxamento",
        "👥 Buscar apoio acadêmico",
        "📱 Reduzir tempo em redes sociais"
    ],
    'Baixo': [
        "🚨 Intervenção imediata necessária",
        "👨‍🏫 Acompanhamento pedagógico intensivo",
        "🧠 Avaliação de saúde mental",
        "📞 Suporte familiar e institucional"
    ]
}

for segment, recs in recommendations.items():
    if segment in df_processed['performance_segment'].values:
        count = len(df_processed[df_processed['performance_segment'] == segment])
        print(f"\n🎯 SEGMENTO {segment.upper()} ({count} estudantes):")
        for rec in recs:
            print(f"  {rec}")

# =============================================================================
# 8.4 MÉTRICAS DE IMPACTO E ROI
# =============================================================================
print("\n📊 MÉTRICAS DE IMPACTO E ROI")
print("=" * 50)

# Calcular impacto potencial das intervenções
current_avg_score = df_processed['exam_score'].mean()
excellent_avg_score = df_processed[df_processed['performance_segment'] == 'Excelente']['exam_score'].mean()

# Simular impacto de melhorias
improvement_scenarios = {
    'Aumentar horas de estudo em 1h/dia': {
        'target_segments': ['Regular', 'Baixo'],
        'expected_improvement': 5,  # pontos
        'cost': 'Baixo'
    },
    'Melhorar saúde mental': {
        'target_segments': ['Regular', 'Baixo'],
        'expected_improvement': 8,  # pontos
        'cost': 'Médio'
    },
    'Reduzir redes sociais em 2h/dia': {
        'target_segments': ['Regular', 'Baixo'],
        'expected_improvement': 6,  # pontos
        'cost': 'Baixo'
    }
}

print("🎯 CENÁRIOS DE MELHORIA:")
print("-" * 40)
for intervention, details in improvement_scenarios.items():
    target_count = sum([len(df_processed[df_processed['performance_segment'] == seg]) 
                       for seg in details['target_segments']])
    potential_impact = target_count * details['expected_improvement']
    
    print(f"\n📈 {intervention}:")
    print(f"  • Estudantes impactados: {target_count}")
    print(f"  • Melhoria esperada: {details['expected_improvement']} pontos")
    print(f"  • Impacto total: {potential_impact} pontos")
    print(f"  • Custo: {details['cost']}")

print(f"\n🎯 MÉTRICAS ATUAIS:")
print(f"  • Nota média atual: {current_avg_score:.1f}")
print(f"  • Nota média dos excelentes: {excellent_avg_score:.1f}")
print(f"  • Potencial de melhoria: {excellent_avg_score - current_avg_score:.1f} pontos")

print("\n✅ Análise de insights e recomendações concluída!")


## 🔗 Análise de Correlações

Esta seção explora as relações entre diferentes variáveis:

### 📈 Tipos de Correlações Analisadas:
- **Correlações Positivas** - Fatores que melhoram o desempenho
- **Correlações Negativas** - Fatores que prejudicam o desempenho
- **Matriz de Correlação** - Visão completa das relações
- **Fatores de Impacto** - Ranking das variáveis mais influentes


In [ ]:
# =============================================================================
# 7. VISUALIZAÇÕES DE CORRELAÇÕES/ ANALISAR BEM ESSE AQ
# =============================================================================

print("📈 MAPA DE CALOR DE CORRELAÇÕES")
print("=" * 50)

# Criar figura com subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Criar matriz de correlação excluindo a coluna 'cluster'
numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
# Excluir a coluna 'cluster' da análise de correlação
numeric_cols = numeric_cols.drop('cluster') if 'cluster' in numeric_cols else numeric_cols
correlation_matrix = df_processed[numeric_cols].corr()

# Mapa de calor completo
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='RdYlBu_r', 
            center=0,
            square=True,
            fmt='.2f',
            cbar_kws={"shrink": .8},
            ax=ax1)
ax1.set_title('Mapa de Calor: Correlações Completas', fontsize=16, fontweight='bold', pad=20)

# Mapa de calor focado no desempenho acadêmico
exam_corr_data = correlation_matrix[['exam_score']].sort_values('exam_score', ascending=False)
sns.heatmap(exam_corr_data, 
            annot=True, 
            cmap='RdYlGn', 
            center=0,
            fmt='.3f',
            cbar_kws={"shrink": .8},
            ax=ax2)
ax2.set_title('Correlações com Desempenho Acadêmico', fontsize=16, fontweight='bold', pad=20)
ax2.set_xlabel('')

plt.tight_layout()
plt.show()


## 📊 Análises Comparativas Detalhadas

Esta seção realiza comparações aprofundadas entre diferentes grupos:

### 🔍 Comparações Realizadas:
- **Por Gênero** - Diferenças comportamentais entre homens e mulheres
- **Por Educação dos Pais** - Impacto do background familiar
- **Por Faixa Etária** - Variações por idade
- **Por Nível de Desempenho** - Características de cada segmento


In [ ]:
# =============================================================================
# 8. ANÁLISES COMPARATIVAS
# =============================================================================

print("📊 ANÁLISE DETALHADA E INSIGHTS")
print("=" * 50)

# Análise de perfis de estudantes
print("🎯 PERFIS DE ESTUDANTES POR DESEMPENHO")
print("=" * 50)

# Estudantes de alto desempenho (top 25%)
high_performers = df_processed[df_processed['performance_category'] == 'Excelente']
print(f"📈 Estudantes de Excelente Desempenho: {len(high_performers)} ({len(high_performers)/len(df_processed)*100:.1f}%)")

print("\n🔍 CARACTERÍSTICAS DOS ESTUDANTES DE ALTO DESEMPENHO:")
print("-" * 60)
print(f"• Média de horas de estudo: {high_performers['study_hours_per_day'].mean():.1f} horas/dia")
print(f"• Média de horas de redes sociais: {high_performers['social_media_hours'].mean():.1f} horas/dia")
print(f"• Média de horas de sono: {high_performers['sleep_hours'].mean():.1f} horas")
print(f"• Média de frequência de exercício: {high_performers['exercise_frequency'].mean():.1f}")
print(f"• Média de saúde mental: {high_performers['mental_health_rating'].mean():.1f}/10")
print(f"• Participação em atividades extracurriculares: {high_performers['extracurricular_participation'].value_counts(normalize=True)['Yes']*100:.1f}%")

# Estudantes de baixo desempenho
low_performers = df_processed[df_processed['performance_category'] == 'Baixo']
print(f"\n📉 Estudantes de Baixo Desempenho: {len(low_performers)} ({len(low_performers)/len(df_processed)*100:.1f}%)")

print("\n🔍 CARACTERÍSTICAS DOS ESTUDANTES DE BAIXO DESEMPENHO:")
print("-" * 60)
print(f"• Média de horas de estudo: {low_performers['study_hours_per_day'].mean():.1f} horas/dia")
print(f"• Média de horas de redes sociais: {low_performers['social_media_hours'].mean():.1f} horas/dia")
print(f"• Média de horas de sono: {low_performers['sleep_hours'].mean():.1f} horas")
print(f"• Média de frequência de exercício: {low_performers['exercise_frequency'].mean():.1f}")
print(f"• Média de saúde mental: {low_performers['mental_health_rating'].mean():.1f}/10")
print(f"• Participação em atividades extracurriculares: {low_performers['extracurricular_participation'].value_counts(normalize=True)['Yes']*100:.1f}%")

# Análise de correlações mais significativas
print("\n🎯 PRINCIPAIS INSIGHTS")
print("=" * 50)

# Criar matriz de correlação excluindo a coluna 'cluster'
numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
# Excluir a coluna 'cluster' da análise de correlação
numeric_cols = numeric_cols.drop('cluster') if 'cluster' in numeric_cols else numeric_cols
correlation_matrix = df_processed[numeric_cols].corr()
exam_correlations = correlation_matrix['exam_score'].drop('exam_score').sort_values(ascending=True)

# Top 3 variáveis com maior impacto positivo
top_3_positive = exam_correlations[exam_correlations.index != 'exam_score'].head(3)
print("📈 TOP 3 FATORES POSITIVOS:")
for i, (var, corr) in enumerate(top_3_positive.items(), 1):
    print(f"{i}. {var}: {corr:.3f}")

# Top 3 variáveis com maior impacto negativo
top_3_negative = exam_correlations[exam_correlations < 0].tail(3)
print("\n📉 TOP 3 FATORES NEGATIVOS:")
for i, (var, corr) in enumerate(top_3_negative.items(), 1):
    print(f"{i}. {var}: {corr:.3f}")

# Análise de diferenças por gênero
print("\n👥 ANÁLISE POR GÊNERO")
print("=" * 30)
gender_analysis = df_processed.groupby('gender')['exam_score'].agg(['count', 'mean', 'std'])
display(gender_analysis.round(2))

# Análise de diferenças por educação dos pais
print("\n👨‍👩‍👧‍👦 ANÁLISE POR EDUCAÇÃO DOS PAIS")
print("=" * 40)
parent_education_analysis = df_processed.groupby('parental_education_level')['exam_score'].agg(['count', 'mean', 'std'])
display(parent_education_analysis.round(2))




## Dashboard:
- **Métricas-Chave** - Indicadores principais de desempenho
- **Distribuição de Performance** - Segmentação dos estudantes
- **Fatores de Impacto** - Variáveis mais influentes
- **Análise de Clusters** - Agrupamento por comportamentos similares


In [ ]:
# =============================================================================
# 10. ANÁLISE DAS TENDÊNCIAS IDENTIFICADAS E SEGMENTAÇÃO DE ESTUDANTES
# =============================================================================

print("📊 ANÁLISE DAS TENDÊNCIAS E SEGMENTAÇÃO DE ESTUDANTES")
print("=" * 70)

# Configurações para visualizações
plt.style.use('default')
sns.set_style("whitegrid")
plt.rcParams.update({
    'figure.figsize': (20, 15),
    'font.size': 12,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12
})

# Paleta de cores
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#7209B7', '#F77F00', '#06FFA5']

# =============================================================================
# 10.1 TENDÊNCIA 1: Exam score tem alta correlação com study hours per day
# =============================================================================

print("📈 TENDÊNCIA 1: Correlação Alta entre Horas de Estudo e Desempenho")
print("=" * 70)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))

# Gráfico 1: Scatter plot com linha de regressão
correlation_study = df_processed['study_hours_per_day'].corr(df_processed['exam_score'])
ax1.scatter(df_processed['study_hours_per_day'], df_processed['exam_score'], 
           alpha=0.6, c=df_processed['mental_health_rating'], cmap='viridis', s=50)
ax1.set_xlabel('Horas de Estudo por Dia')
ax1.set_ylabel('Nota do Exame')
ax1.set_title(f'Relação Horas de Estudo vs Desempenho\nCorrelação: {correlation_study:.3f}', 
              fontweight='bold', fontsize=14)

# Linha de regressão
z = np.polyfit(df_processed['study_hours_per_day'], df_processed['exam_score'], 1)
p = np.poly1d(z)
ax1.plot(df_processed['study_hours_per_day'], p(df_processed['study_hours_per_day']), 
         "r--", alpha=0.8, linewidth=3)

# Segmentação por horas de estudo
def categorize_study_intensity(hours):
    if hours <= 2: return 'Baixo (≤2h)'
    elif hours <= 4: return 'Moderado (2-4h)'
    elif hours <= 6: return 'Alto (4-6h)'
    else: return 'Muito Alto (>6h)'

df_processed['study_intensity'] = df_processed['study_hours_per_day'].apply(categorize_study_intensity)

# Gráfico 2: Boxplot por intensidade de estudo
study_groups = df_processed.groupby('study_intensity')['exam_score'].apply(list)
study_labels = list(study_groups.index)
study_data = list(study_groups.values)

box_plot = ax2.boxplot(study_data, labels=study_labels, patch_artist=True)
for patch, color in zip(box_plot['boxes'], colors[:len(study_labels)]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax2.set_ylabel('Nota do Exame')
ax2.set_title('Desempenho por Intensidade de Estudo', fontweight='bold', fontsize=14)
ax2.tick_params(axis='x', rotation=45)

# Análise estatística por grupo
print("\n📊 ANÁLISE POR GRUPOS DE INTENSIDADE DE ESTUDO:")
print("-" * 60)
for group in study_labels:
    group_data = df_processed[df_processed['study_intensity'] == group]
    print(f"🎯 {group}:")
    print(f"  • Quantidade: {len(group_data)} estudantes ({len(group_data)/len(df_processed)*100:.1f}%)")
    print(f"  • Nota média: {group_data['exam_score'].mean():.1f} ± {group_data['exam_score'].std():.1f}")
    print(f"  • Horas médias de estudo: {group_data['study_hours_per_day'].mean():.1f}h/dia")

# =============================================================================
# 10.2 TENDÊNCIA 2: Exam score é negativamente afetado por uso redes sociais e Netflix
# =============================================================================

print("\n📉 TENDÊNCIA 2: Impacto Negativo das Redes Sociais e Netflix")
print("=" * 70)

# Gráfico 3: Scatter plot redes sociais vs desempenho
correlation_social = df_processed['social_media_hours'].corr(df_processed['exam_score'])
ax3.scatter(df_processed['social_media_hours'], df_processed['exam_score'], 
           alpha=0.6, c=df_processed['study_hours_per_day'], cmap='plasma', s=50)
ax3.set_xlabel('Horas de Redes Sociais por Dia')
ax3.set_ylabel('Nota do Exame')
ax3.set_title(f'Relação Redes Sociais vs Desempenho\nCorrelação: {correlation_social:.3f}', 
              fontweight='bold', fontsize=14)

# Linha de regressão
z_social = np.polyfit(df_processed['social_media_hours'], df_processed['exam_score'], 1)
p_social = np.poly1d(z_social)
ax3.plot(df_processed['social_media_hours'], p_social(df_processed['social_media_hours']), 
         "r--", alpha=0.8, linewidth=3)

# Segmentação por uso de redes sociais
def categorize_social_usage(hours):
    if hours <= 1: return 'Baixo (≤1h)'
    elif hours <= 3: return 'Moderado (1-3h)'
    elif hours <= 5: return 'Alto (3-5h)'
    else: return 'Muito Alto (>5h)'

df_processed['social_usage'] = df_processed['social_media_hours'].apply(categorize_social_usage)

# Gráfico 4: Boxplot por uso de redes sociais
social_groups = df_processed.groupby('social_usage')['exam_score'].apply(list)
social_labels = list(social_groups.index)
social_data = list(social_groups.values)

box_plot_social = ax4.boxplot(social_data, labels=social_labels, patch_artist=True)
for patch, color in zip(box_plot_social['boxes'], colors[:len(social_labels)]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax4.set_ylabel('Nota do Exame')
ax4.set_title('Desempenho por Uso de Redes Sociais', fontweight='bold', fontsize=14)
ax4.tick_params(axis='x', rotation=45)

# Análise estatística por grupo de redes sociais
print("\n📊 ANÁLISE POR GRUPOS DE USO DE REDES SOCIAIS:")
print("-" * 60)
for group in social_labels:
    group_data = df_processed[df_processed['social_usage'] == group]
    print(f"🎯 {group}:")
    print(f"  • Quantidade: {len(group_data)} estudantes ({len(group_data)/len(df_processed)*100:.1f}%)")
    print(f"  • Nota média: {group_data['exam_score'].mean():.1f} ± {group_data['exam_score'].std():.1f}")
    print(f"  • Horas médias de redes sociais: {group_data['social_media_hours'].mean():.1f}h/dia")

plt.tight_layout()
plt.show()

# =============================================================================
# 10.3 TENDÊNCIA 3: Exam score é levemente afetado por boas horas de sono e exercícios
# =============================================================================

print("\n😴 TENDÊNCIA 3: Impacto do Sono e Exercícios no Desempenho")
print("=" * 70)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))

# Gráfico 1: Sono vs Desempenho
correlation_sleep = df_processed['sleep_hours'].corr(df_processed['exam_score'])
ax1.scatter(df_processed['sleep_hours'], df_processed['exam_score'], 
           alpha=0.6, c=df_processed['mental_health_rating'], cmap='coolwarm', s=50)
ax1.set_xlabel('Horas de Sono por Noite')
ax1.set_ylabel('Nota do Exame')
ax1.set_title(f'Relação Sono vs Desempenho\nCorrelação: {correlation_sleep:.3f}', 
              fontweight='bold', fontsize=14)

# Linha de regressão
z_sleep = np.polyfit(df_processed['sleep_hours'], df_processed['exam_score'], 1)
p_sleep = np.poly1d(z_sleep)
ax1.plot(df_processed['sleep_hours'], p_sleep(df_processed['sleep_hours']), 
         "r--", alpha=0.8, linewidth=3)

# Segmentação por qualidade do sono
def categorize_sleep_quality(hours):
    if hours < 6: return 'Insuficiente (<6h)'
    elif hours <= 7: return 'Adequado (6-7h)'
    elif hours <= 9: return 'Ideal (7-9h)'
    else: return 'Excessivo (>9h)'

df_processed['sleep_quality'] = df_processed['sleep_hours'].apply(categorize_sleep_quality)

# Gráfico 2: Boxplot por qualidade do sono
sleep_groups = df_processed.groupby('sleep_quality')['exam_score'].apply(list)
sleep_labels = list(sleep_groups.index)
sleep_data = list(sleep_groups.values)

box_plot_sleep = ax2.boxplot(sleep_data, labels=sleep_labels, patch_artist=True)
for patch, color in zip(box_plot_sleep['boxes'], colors[:len(sleep_labels)]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax2.set_ylabel('Nota do Exame')
ax2.set_title('Desempenho por Qualidade do Sono', fontweight='bold', fontsize=14)
ax2.tick_params(axis='x', rotation=45)

# Gráfico 3: Exercícios vs Desempenho
correlation_exercise = df_processed['exercise_frequency'].corr(df_processed['exam_score'])
ax3.scatter(df_processed['exercise_frequency'], df_processed['exam_score'], 
           alpha=0.6, c=df_processed['sleep_hours'], cmap='spring', s=50)
ax3.set_xlabel('Frequência de Exercícios (vezes/semana)')
ax3.set_ylabel('Nota do Exame')
ax3.set_title(f'Relação Exercícios vs Desempenho\nCorrelação: {correlation_exercise:.3f}', 
              fontweight='bold', fontsize=14)

# Linha de regressão
z_exercise = np.polyfit(df_processed['exercise_frequency'], df_processed['exam_score'], 1)
p_exercise = np.poly1d(z_exercise)
ax3.plot(df_processed['exercise_frequency'], p_exercise(df_processed['exercise_frequency']), 
         "r--", alpha=0.8, linewidth=3)

# Segmentação por frequência de exercícios
def categorize_exercise_frequency(freq):
    if freq <= 1: return 'Sedentário (≤1x)'
    elif freq <= 3: return 'Ocasional (1-3x)'
    elif freq <= 5: return 'Regular (3-5x)'
    else: return 'Ativo (>5x)'

df_processed['exercise_level'] = df_processed['exercise_frequency'].apply(categorize_exercise_frequency)

# Gráfico 4: Boxplot por nível de exercício
exercise_groups = df_processed.groupby('exercise_level')['exam_score'].apply(list)
exercise_labels = list(exercise_groups.index)
exercise_data = list(exercise_groups.values)

box_plot_exercise = ax4.boxplot(exercise_data, labels=exercise_labels, patch_artist=True)
for patch, color in zip(box_plot_exercise['boxes'], colors[:len(exercise_labels)]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax4.set_ylabel('Nota do Exame')
ax4.set_title('Desempenho por Nível de Exercício', fontweight='bold', fontsize=14)
ax4.tick_params(axis='x', rotation=45)

# Análise estatística por grupos de sono e exercício
print("\n📊 ANÁLISE POR GRUPOS DE SONO:")
print("-" * 50)
for group in sleep_labels:
    group_data = df_processed[df_processed['sleep_quality'] == group]
    print(f"😴 {group}:")
    print(f"  • Quantidade: {len(group_data)} estudantes ({len(group_data)/len(df_processed)*100:.1f}%)")
    print(f"  • Nota média: {group_data['exam_score'].mean():.1f} ± {group_data['exam_score'].std():.1f}")
    print(f"  • Horas médias de sono: {group_data['sleep_hours'].mean():.1f}h")

print("\n📊 ANÁLISE POR GRUPOS DE EXERCÍCIO:")
print("-" * 50)
for group in exercise_labels:
    group_data = df_processed[df_processed['exercise_level'] == group]
    print(f"🏃 {group}:")
    print(f"  • Quantidade: {len(group_data)} estudantes ({len(group_data)/len(df_processed)*100:.1f}%)")
    print(f"  • Nota média: {group_data['exam_score'].mean():.1f} ± {group_data['exam_score'].std():.1f}")
    print(f"  • Frequência média de exercício: {group_data['exercise_frequency'].mean():.1f}x/semana")

plt.tight_layout()
plt.show()

# =============================================================================
# 10.4 TENDÊNCIA 4: Exam Score tem relação direta com study_screen_ratio
# =============================================================================

print("\n📱 TENDÊNCIA 4: Relação com Razão Estudo/Tela")
print("=" * 70)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))

# Gráfico 1: Razão estudo/tela vs desempenho
correlation_ratio = df_processed['study_screen_ratio'].corr(df_processed['exam_score'])
ax1.scatter(df_processed['study_screen_ratio'], df_processed['exam_score'], 
           alpha=0.6, c=df_processed['mental_health_rating'], cmap='viridis', s=50)
ax1.set_xlabel('Razão Estudo/Tela (study_screen_ratio)')
ax1.set_ylabel('Nota do Exame')
ax1.set_title(f'Relação Razão Estudo/Tela vs Desempenho\nCorrelação: {correlation_ratio:.3f}', 
              fontweight='bold', fontsize=14)

# Linha de regressão
z_ratio = np.polyfit(df_processed['study_screen_ratio'], df_processed['exam_score'], 1)
p_ratio = np.poly1d(z_ratio)
ax1.plot(df_processed['study_screen_ratio'], p_ratio(df_processed['study_screen_ratio']), 
         "r--", alpha=0.8, linewidth=3)

# Segmentação por razão estudo/tela
def categorize_study_screen_ratio(ratio):
    if ratio <= 0.5: return 'Baixa (≤0.5)'
    elif ratio <= 1.0: return 'Moderada (0.5-1.0)'
    elif ratio <= 2.0: return 'Alta (1.0-2.0)'
    else: return 'Muito Alta (>2.0)'

df_processed['study_screen_level'] = df_processed['study_screen_ratio'].apply(categorize_study_screen_ratio)

# Gráfico 2: Boxplot por nível de razão estudo/tela
ratio_groups = df_processed.groupby('study_screen_level')['exam_score'].apply(list)
ratio_labels = list(ratio_groups.index)
ratio_data = list(ratio_groups.values)

box_plot_ratio = ax2.boxplot(ratio_data, labels=ratio_labels, patch_artist=True)
for patch, color in zip(box_plot_ratio['boxes'], colors[:len(ratio_labels)]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax2.set_ylabel('Nota do Exame')
ax2.set_title('Desempenho por Nível de Razão Estudo/Tela', fontweight='bold', fontsize=14)
ax2.tick_params(axis='x', rotation=45)

# Gráfico 3: Análise combinada - Estudo vs Redes Sociais
ax3.scatter(df_processed['study_hours_per_day'], df_processed['social_media_hours'], 
           c=df_processed['exam_score'], cmap='RdYlGn', s=60, alpha=0.7)
ax3.set_xlabel('Horas de Estudo por Dia')
ax3.set_ylabel('Horas de Redes Sociais por Dia')
ax3.set_title('Estudo vs Redes Sociais (Cor: Desempenho)', fontweight='bold', fontsize=14)

# Adicionar contornos
from scipy.stats import gaussian_kde
xy = np.vstack([df_processed['study_hours_per_day'], df_processed['social_media_hours']])
density = gaussian_kde(xy)
x_range = np.linspace(df_processed['study_hours_per_day'].min(), df_processed['study_hours_per_day'].max(), 50)
y_range = np.linspace(df_processed['social_media_hours'].min(), df_processed['social_media_hours'].max(), 50)
X, Y = np.meshgrid(x_range, y_range)
positions = np.vstack([X.ravel(), Y.ravel()])
Z = np.reshape(density(positions).T, X.shape)
ax3.contour(X, Y, Z, levels=5, colors='black', alpha=0.6, linewidths=1)

# Gráfico 4: Análise de perfis combinados
# Criar perfis baseados em múltiplas variáveis
def create_student_profile(row):
    study_hours = row['study_hours_per_day']
    social_hours = row['social_media_hours']
    sleep_hours = row['sleep_hours']
    exercise_freq = row['exercise_frequency']
    
    if study_hours >= 5 and social_hours <= 2 and sleep_hours >= 7:
        return 'Estudante Ideal'
    elif study_hours >= 4 and social_hours <= 3:
        return 'Estudante Dedicado'
    elif study_hours >= 3 and social_hours <= 4:
        return 'Estudante Moderado'
    elif social_hours >= 5:
        return 'Estudante Distraído'
    else:
        return 'Estudante Irregular'

df_processed['student_profile'] = df_processed.apply(create_student_profile, axis=1)

# Contar perfis
profile_counts = df_processed['student_profile'].value_counts()
profile_performance = df_processed.groupby('student_profile')['exam_score'].mean().sort_values(ascending=False)

# Gráfico de barras dos perfis
bars = ax4.bar(range(len(profile_counts)), profile_counts.values, 
               color=colors[:len(profile_counts)], alpha=0.7)
ax4.set_xlabel('Perfil do Estudante')
ax4.set_ylabel('Número de Estudantes')
ax4.set_title('Distribuição por Perfil de Estudante', fontweight='bold', fontsize=14)
ax4.set_xticks(range(len(profile_counts)))
ax4.set_xticklabels(profile_counts.index, rotation=45, ha='right')

# Adicionar valores de performance nas barras
for i, (bar, perf) in enumerate(zip(bars, profile_performance.values)):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
             f'{perf:.1f}', ha='center', va='bottom', fontweight='bold')

# Análise estatística por grupos de razão estudo/tela
print("\n📊 ANÁLISE POR GRUPOS DE RAZÃO ESTUDO/TELA:")
print("-" * 60)
for group in ratio_labels:
    group_data = df_processed[df_processed['study_screen_level'] == group]
    print(f"📱 {group}:")
    print(f"  • Quantidade: {len(group_data)} estudantes ({len(group_data)/len(df_processed)*100:.1f}%)")
    print(f"  • Nota média: {group_data['exam_score'].mean():.1f} ± {group_data['exam_score'].std():.1f}")
    print(f"  • Razão média: {group_data['study_screen_ratio'].mean():.2f}")

# Análise por perfis de estudantes
print("\n📊 ANÁLISE POR PERFIS DE ESTUDANTES:")
print("-" * 50)
for profile in profile_performance.index:
    profile_data = df_processed[df_processed['student_profile'] == profile]
    print(f"👤 {profile}:")
    print(f"  • Quantidade: {len(profile_data)} estudantes ({len(profile_data)/len(df_processed)*100:.1f}%)")
    print(f"  • Nota média: {profile_data['exam_score'].mean():.1f} ± {profile_data['exam_score'].std():.1f}")
    print(f"  • Horas de estudo: {profile_data['study_hours_per_day'].mean():.1f}h/dia")
    print(f"  • Horas de redes sociais: {profile_data['social_media_hours'].mean():.1f}h/dia")

plt.tight_layout()
plt.show()

# =============================================================================
# 10.5 TENDÊNCIA 5: Gênero tem relação mas bem baixa, mulheres tem nota um pouquinho mais alto
# =============================================================================

print("\n👥 TENDÊNCIA 5: Análise por Gênero")
print("=" * 70)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))

# Gráfico 1: Distribuição de notas por gênero
gender_performance = df_processed.groupby('gender')['exam_score'].apply(list)
gender_labels = list(gender_performance.index)
gender_data = list(gender_performance.values)

box_plot_gender = ax1.boxplot(gender_data, labels=gender_labels, patch_artist=True)
for patch, color in zip(box_plot_gender['boxes'], colors[:len(gender_labels)]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax1.set_ylabel('Nota do Exame')
ax1.set_title('Distribuição de Notas por Gênero', fontweight='bold', fontsize=14)

# Análise estatística por gênero
print("\n📊 ANÁLISE POR GÊNERO:")
print("-" * 40)
for gender in gender_labels:
    gender_data = df_processed[df_processed['gender'] == gender]
    print(f"👤 {gender}:")
    print(f"  • Quantidade: {len(gender_data)} estudantes ({len(gender_data)/len(df_processed)*100:.1f}%)")
    print(f"  • Nota média: {gender_data['exam_score'].mean():.1f} ± {gender_data['exam_score'].std():.1f}")
    print(f"  • Horas de estudo: {gender_data['study_hours_per_day'].mean():.1f}h/dia")
    print(f"  • Horas de redes sociais: {gender_data['social_media_hours'].mean():.1f}h/dia")
    print(f"  • Saúde mental: {gender_data['mental_health_rating'].mean():.1f}/10")

# Gráfico 2: Análise de correlações por gênero
correlations_by_gender = {}
for gender in gender_labels:
    gender_data = df_processed[df_processed['gender'] == gender]
    correlations_by_gender[gender] = {
        'study_hours': gender_data['study_hours_per_day'].corr(gender_data['exam_score']),
        'social_media': gender_data['social_media_hours'].corr(gender_data['exam_score']),
        'sleep': gender_data['sleep_hours'].corr(gender_data['exam_score']),
        'exercise': gender_data['exercise_frequency'].corr(gender_data['exam_score'])
    }

# Gráfico de barras das correlações por gênero
correlation_vars = ['study_hours', 'social_media', 'sleep', 'exercise']
x = np.arange(len(correlation_vars))
width = 0.35

for i, gender in enumerate(gender_labels):
    correlations = [correlations_by_gender[gender][var] for var in correlation_vars]
    ax2.bar(x + i*width, correlations, width, label=gender, alpha=0.7)

ax2.set_xlabel('Variáveis')
ax2.set_ylabel('Correlação com Desempenho')
ax2.set_title('Correlações por Gênero', fontweight='bold', fontsize=14)
ax2.set_xticks(x + width/2)
ax2.set_xticklabels([var.replace('_', ' ').title() for var in correlation_vars])
ax2.legend()
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Gráfico 3: Análise de distribuição de performance por gênero
performance_by_gender = pd.crosstab(df_processed['gender'], df_processed['performance_category'])
performance_by_gender_pct = performance_by_gender.div(performance_by_gender.sum(axis=1), axis=0) * 100

performance_by_gender_pct.plot(kind='bar', ax=ax3, color=colors[:4])
ax3.set_title('Distribuição de Performance por Gênero', fontweight='bold', fontsize=14)
ax3.set_xlabel('Gênero')
ax3.set_ylabel('Percentual (%)')
ax3.legend(title='Performance', bbox_to_anchor=(1.05, 1), loc='upper left')
ax3.tick_params(axis='x', rotation=0)

# Gráfico 4: Análise de hábitos por gênero
habits_by_gender = df_processed.groupby('gender')[['study_hours_per_day', 'social_media_hours', 
                                                   'sleep_hours', 'exercise_frequency']].mean()

habits_by_gender.plot(kind='bar', ax=ax4, color=colors[:4])
ax4.set_title('Hábitos por Gênero', fontweight='bold', fontsize=14)
ax4.set_xlabel('Gênero')
ax4.set_ylabel('Valor Médio')
ax4.legend(title='Hábitos', bbox_to_anchor=(1.05, 1), loc='upper left')
ax4.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

# =============================================================================
# 10.6 RESUMO DAS TENDÊNCIAS E SEGMENTAÇÃO FINAL
# =============================================================================

print("\n🎯 RESUMO DAS TENDÊNCIAS E SEGMENTAÇÃO FINAL")
print("=" * 70)

# Criar segmentação final baseada em múltiplas variáveis
def create_final_segments(row):
    score = row['exam_score']
    study_hours = row['study_hours_per_day']
    social_hours = row['social_media_hours']
    sleep_hours = row['sleep_hours']
    exercise_freq = row['exercise_frequency']
    mental_health = row['mental_health_rating']
    
    # Segmentação baseada em performance e hábitos
    if score >= 85 and study_hours >= 4 and social_hours <= 3:
        return 'Estrelas Acadêmicas'
    elif score >= 70 and study_hours >= 3 and social_hours <= 4:
        return 'Estudantes Dedicados'
    elif score >= 50 and study_hours >= 2:
        return 'Estudantes em Desenvolvimento'
    elif social_hours >= 5 or study_hours < 2:
        return 'Estudantes em Risco'
    else:
        return 'Estudantes Regulares'

df_processed['final_segment'] = df_processed.apply(create_final_segments, axis=1)

# Análise dos segmentos finais
print("\n📊 SEGMENTAÇÃO FINAL DOS ESTUDANTES:")
print("-" * 60)
segment_analysis = df_processed.groupby('final_segment').agg({
    'exam_score': ['count', 'mean', 'std'],
    'study_hours_per_day': 'mean',
    'social_media_hours': 'mean',
    'sleep_hours': 'mean',
    'exercise_frequency': 'mean',
    'mental_health_rating': 'mean'
}).round(2)

for segment in segment_analysis.index:
    data = segment_analysis.loc[segment]
    count = data[('exam_score', 'count')]
    mean_score = data[('exam_score', 'mean')]
    std_score = data[('exam_score', 'std')]
    
    print(f"\n🎯 {segment.upper()}:")
    print(f"  • Quantidade: {count} estudantes ({count/len(df_processed)*100:.1f}%)")
    print(f"  • Nota média: {mean_score:.1f} ± {std_score:.1f}")
    print(f"  • Horas de estudo: {data[('study_hours_per_day', 'mean')]:.1f}h/dia")
    print(f"  • Horas de redes sociais: {data[('social_media_hours', 'mean')]:.1f}h/dia")
    print(f"  • Horas de sono: {data[('sleep_hours', 'mean')]:.1f}h")
    print(f"  • Frequência de exercício: {data[('exercise_frequency', 'mean')]:.1f}x/semana")
    print(f"  • Saúde mental: {data[('mental_health_rating', 'mean')]:.1f}/10")

# Correlações finais
print(f"\n📈 CORRELAÇÕES FINAIS DAS TENDÊNCIAS:")
print("-" * 50)
print(f"1. Horas de Estudo vs Desempenho: {df_processed['study_hours_per_day'].corr(df_processed['exam_score']):.3f}")
print(f"2. Redes Sociais vs Desempenho: {df_processed['social_media_hours'].corr(df_processed['exam_score']):.3f}")
print(f"3. Netflix vs Desempenho: {df_processed['netflix_hours'].corr(df_processed['exam_score']):.3f}")
print(f"4. Sono vs Desempenho: {df_processed['sleep_hours'].corr(df_processed['exam_score']):.3f}")
print(f"5. Exercício vs Desempenho: {df_processed['exercise_frequency'].corr(df_processed['exam_score']):.3f}")
print(f"6. Razão Estudo/Tela vs Desempenho: {df_processed['study_screen_ratio'].corr(df_processed['exam_score']):.3f}")

# Análise por gênero
gender_correlation = df_processed.groupby('gender')['exam_score'].mean()
print(f"\n👥 DIFERENÇA POR GÊNERO:")
print("-" * 30)
for gender in gender_correlation.index:
    print(f"  • {gender}: {gender_correlation[gender]:.1f} pontos")
print(f"  • Diferença: {gender_correlation.max() - gender_correlation.min():.1f} pontos")

print(f"\n✅ ANÁLISE DAS TENDÊNCIAS CONCLUÍDA!")
print(f"📊 Total de segmentos criados: {len(df_processed['final_segment'].unique())}")
print(f"🎯 Estudantes analisados: {len(df_processed)}")
print(f"📈 Variáveis analisadas: {df_processed.shape[1]}")


## 📈 Análise de Tendências e Segmentação

Esta seção identifica padrões comportamentais e segmenta os estudantes:

### 🎯 Análises Realizadas:
- **Segmentação por Performance** - Agrupamento por níveis de desempenho
- **Identificação de Padrões** - Comportamentos comuns em cada grupo
- **Análise de Risco** - Estudantes em situação vulnerável
- **Recomendações Personalizadas** - Sugestões específicas por segmento


In [ ]:
# =============================================================================
# HEATMAP COMPLETO - TODAS AS VARIÁVEIS
# =============================================================================

print("📊 HEATMAP COMPLETO - TODAS AS VARIÁVEIS")
print("=" * 60)

# Configurações para visualização
plt.style.use('default')
sns.set_style("whitegrid")
plt.rcParams.update({
    'figure.figsize': (20, 16),
    'font.size': 12,
    'axes.titlesize': 18,
    'axes.labelsize': 14,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 12
})

# Selecionar todas as variáveis numéricas do dataset original + novas variáveis
numeric_vars_original = [
    'age', 'study_hours_per_day', 'social_media_hours', 'netflix_hours',
    'attendance_percentage', 'sleep_hours', 'exercise_frequency', 
    'mental_health_rating', 'exam_score'
]

# Adicionar as 4 novas variáveis criadas
new_vars = ['total_leisure_hours', 'available_study_time', 'study_screen_ratio', 'wellness_index']

# Variáveis categóricas que serão transformadas
categorical_vars = [
    'gender', 'part_time_job', 'diet_quality', 'parental_education_level',
    'internet_quality', 'extracurricular_participation'
]

# Combinar todas as variáveis
all_numeric_vars = numeric_vars_original + new_vars

# Criar DataFrame com todas as variáveis numéricas
df_heatmap = df_processed[all_numeric_vars].copy()

# =============================================================================
# TRANSFORMAÇÃO DE VARIÁVEIS CATEGÓRICAS
# =============================================================================

print("🔄 TRANSFORMANDO VARIÁVEIS CATEGÓRICAS EM NUMÉRICAS")
print("=" * 60)

# Transformar variáveis categóricas em numéricas
for var in categorical_vars:
    if var in df_processed.columns:
        print(f"📊 Transformando {var}...")
        
        # Usar LabelEncoder para transformar categóricas em numéricas
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        
        # Aplicar transformação
        df_heatmap[f'{var}_encoded'] = le.fit_transform(df_processed[var].astype(str))
        
        # Mostrar mapeamento
        unique_values = df_processed[var].unique()
        encoded_values = le.transform(unique_values.astype(str))
        mapping = dict(zip(unique_values, encoded_values))
        print(f"   Mapeamento: {mapping}")

# Atualizar lista de variáveis para incluir as transformadas
encoded_vars = [f'{var}_encoded' for var in categorical_vars if var in df_processed.columns]
all_vars_for_heatmap = all_numeric_vars + encoded_vars

print(f"\n✅ Transformação concluída!")
print(f"📊 Total de variáveis no heatmap: {len(all_vars_for_heatmap)}")
print(f"   • Variáveis numéricas originais: {len(numeric_vars_original)}")
print(f"   • Novas variáveis criadas: {len(new_vars)}")
print(f"   • Variáveis categóricas transformadas: {len(encoded_vars)}")

# Calcular matriz de correlação com todas as variáveis
correlation_matrix_complete = df_heatmap[all_vars_for_heatmap].corr()

# Criar figura com subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 10))

# =============================================================================
# HEATMAP 1: MATRIZ COMPLETA COM MÁSCARA
# =============================================================================

# Criar máscara para mostrar apenas metade da matriz (evitar duplicação)
mask = np.triu(np.ones_like(correlation_matrix_complete, dtype=bool))

# Heatmap completo
sns.heatmap(correlation_matrix_complete, 
            mask=mask,
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            square=True,
            fmt='.2f',
            cbar_kws={"shrink": .8},
            ax=ax1,
            linewidths=0.5,
            annot_kws={'size': 8})

ax1.set_title('Matriz de Correlações Completa\n(Todas as Variáveis)', 
              fontsize=16, fontweight='bold', pad=20)

# =============================================================================
# HEATMAP 2: FOCO NO EXAM_SCORE
# =============================================================================

# Selecionar apenas correlações com exam_score
exam_correlations = correlation_matrix_complete['exam_score'].drop('exam_score').sort_values(ascending=False)

# Criar DataFrame para o heatmap focado
exam_corr_df = pd.DataFrame({
    'exam_score': exam_correlations
}).T

# Heatmap focado no desempenho
sns.heatmap(exam_corr_df, 
            annot=True, 
            cmap='RdYlGn', 
            center=0,
            fmt='.3f',
            cbar_kws={"shrink": .8},
            ax=ax2,
            linewidths=0.5,
            annot_kws={'size': 10})

ax2.set_title('Correlações com Desempenho Acadêmico\n(Exam Score)', 
              fontsize=16, fontweight='bold', pad=20)
ax2.set_xlabel('')

# Ajustar layout
plt.tight_layout()
plt.show()

# =============================================================================
# ANÁLISE DETALHADA DAS CORRELAÇÕES
# =============================================================================

print("\n📈 ANÁLISE DETALHADA DAS CORRELAÇÕES")
print("=" * 60)

# Separar correlações por tipo
positive_correlations = exam_correlations[exam_correlations > 0].sort_values(ascending=False)
negative_correlations = exam_correlations[exam_correlations < 0].sort_values(ascending=True)

print("\n🟢 CORRELAÇÕES POSITIVAS (Fatores que Melhoram o Desempenho):")
print("-" * 70)
for var, corr in positive_correlations.items():
    strength = "Muito Forte" if abs(corr) > 0.7 else "Forte" if abs(corr) > 0.5 else "Moderada" if abs(corr) > 0.3 else "Fraca"
    print(f"📈 {var:25} | {corr:6.3f} | {strength}")

print("\n🔴 CORRELAÇÕES NEGATIVAS (Fatores que Reduzem o Desempenho):")
print("-" * 70)
for var, corr in negative_correlations.items():
    strength = "Muito Forte" if abs(corr) > 0.7 else "Forte" if abs(corr) > 0.5 else "Moderada" if abs(corr) > 0.3 else "Fraca"
    print(f"📉 {var:25} | {corr:6.3f} | {strength}")

# =============================================================================
# ANÁLISE DAS NOVAS VARIÁVEIS E CATEGÓRICAS TRANSFORMADAS
# =============================================================================

print("\n🆕 ANÁLISE DAS NOVAS VARIÁVEIS CRIADAS:")
print("=" * 60)

new_vars_correlations = exam_correlations[new_vars]
for var in new_vars:
    corr = new_vars_correlations[var]
    strength = "Muito Forte" if abs(corr) > 0.7 else "Forte" if abs(corr) > 0.5 else "Moderada" if abs(corr) > 0.3 else "Fraca"
    direction = "📈" if corr > 0 else "📉"
    print(f"{direction} {var:25} | {corr:6.3f} | {strength}")

print("\n🏷️ ANÁLISE DAS VARIÁVEIS CATEGÓRICAS TRANSFORMADAS:")
print("=" * 60)

encoded_vars_correlations = exam_correlations[encoded_vars]
for var in encoded_vars:
    corr = encoded_vars_correlations[var]
    strength = "Muito Forte" if abs(corr) > 0.7 else "Forte" if abs(corr) > 0.5 else "Moderada" if abs(corr) > 0.3 else "Fraca"
    direction = "📈" if corr > 0 else "📉"
    original_var = var.replace('_encoded', '')
    print(f"{direction} {var:25} | {corr:6.3f} | {strength} ({original_var})")

# =============================================================================
# RANKING DE IMPORTÂNCIA
# =============================================================================

print("\n🏆 RANKING DE IMPORTÂNCIA DAS VARIÁVEIS:")
print("=" * 60)

# Criar ranking por valor absoluto da correlação
importance_ranking = exam_correlations.abs().sort_values(ascending=False)

for i, (var, abs_corr) in enumerate(importance_ranking.items(), 1):
    original_corr = exam_correlations[var]
    direction = "📈" if original_corr > 0 else "📉"
    strength = "Muito Forte" if abs_corr > 0.7 else "Forte" if abs_corr > 0.5 else "Moderada" if abs_corr > 0.3 else "Fraca"
    print(f"{i:2d}. {direction} {var:25} | {original_corr:6.3f} | {strength}")

# =============================================================================
# ESTATÍSTICAS RESUMIDAS
# =============================================================================

print("\n📊 ESTATÍSTICAS RESUMIDAS:")
print("=" * 60)
print(f"• Total de variáveis analisadas: {len(all_vars_for_heatmap)}")
print(f"• Variáveis numéricas originais: {len(numeric_vars_original)}")
print(f"• Novas variáveis criadas: {len(new_vars)}")
print(f"• Variáveis categóricas transformadas: {len(encoded_vars)}")
print(f"• Correlações positivas: {len(positive_correlations)}")
print(f"• Correlações negativas: {len(negative_correlations)}")
print(f"• Correlação mais forte: {exam_correlations.abs().max():.3f}")
print(f"• Variável mais importante: {exam_correlations.abs().idxmax()}")

# =============================================================================
# INSIGHTS PRINCIPAIS
# =============================================================================

print("\n💡 INSIGHTS PRINCIPAIS DO HEATMAP:")
print("=" * 60)

top_3_positive = positive_correlations.head(3)
top_3_negative = negative_correlations.head(3)

print("🟢 TOP 3 FATORES POSITIVOS:")
for i, (var, corr) in enumerate(top_3_positive.items(), 1):
    print(f"  {i}. {var}: {corr:.3f}")

print("\n🔴 TOP 3 FATORES NEGATIVOS:")
for i, (var, corr) in enumerate(top_3_negative.items(), 1):
    print(f"  {i}. {var}: {corr:.3f}")

print("\n🆕 IMPACTO DAS NOVAS VARIÁVEIS:")
for var in new_vars:
    corr = exam_correlations[var]
    rank = list(importance_ranking.index).index(var) + 1
    print(f"  • {var}: {corr:.3f} (Rank #{rank})")

print("\n🏷️ IMPACTO DAS VARIÁVEIS CATEGÓRICAS TRANSFORMADAS:")
for var in encoded_vars:
    corr = exam_correlations[var]
    rank = list(importance_ranking.index).index(var) + 1
    original_var = var.replace('_encoded', '')
    print(f"  • {original_var}: {corr:.3f} (Rank #{rank})")

print(f"\n✅ HEATMAP COMPLETO GERADO COM SUCESSO!")
print(f"📊 Análise de {len(all_vars_for_heatmap)} variáveis concluída!")
print(f"   • {len(numeric_vars_original)} numéricas originais")
print(f"   • {len(new_vars)} novas variáveis criadas") 
print(f"   • {len(encoded_vars)} categóricas transformadas")
